## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [2]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_53.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_53.csv')

### 인코딩 

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

train_data = TabularDataset(IVF_train)
test_data = TabularDataset(IVF_test)

label = '임신_성공_여부'
eval_metric = 'roc_auc'

In [4]:
from autogluon.tabular import TabularPredictor

# 시간 제한 설정 
time_limit =  1 * 10 * 60

# # GPU를 사용할 수 없는 모델을 제외하도록 설정
# exclude_model_types = [
#     'KNN',  # K-Nearest Neighbors
#     'RF',   # Random Forest
#     'XT',   # Extra Trees
#     'LR',   # Linear Regression
#     'NN'    # Tabular Neural Network
# ]

# TabularPredictor 객체 생성 및 학습
predictor = TabularPredictor(
    label=label,
    eval_metric=eval_metric,
    path='AutogluonModels/ag-20250224_code53_IVF'  # 모델 저장 경로
).fit(
    train_data,
    presets='best_quality',  # 'best_quality', 'medium_quality', 'good_quality' 등의 프리셋 설정
    # num_stack_levels=0,  # 스택 레벨 설정 / dynamic_stacking=True(디폴트)인 경우 무시
    num_bag_folds=5,  # 배깅 설정
    time_limit=time_limit,  # 시간 제한 설정
    # num_gpus=1,  # GPU 사용 설정
    # excluded_model_types=exclude_model_types  # 제외할 모델 유형 설정
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Memory Avail:       10.48 GB / 15.86 GB (66.1%)
Disk Space Avail:   183.40 GB / 476.30 GB (38.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up t

In [9]:
print(predictor.leaderboard(silent = True))

                      model  score_val eval_metric  pred_time_val    fit_time  \
0       WeightedEnsemble_L3   0.751401     roc_auc     190.925792  201.868731   
1   RandomForestGini_BAG_L2   0.750244     roc_auc     177.835828  155.321147   
2   RandomForestEntr_BAG_L2   0.748814     roc_auc     174.566615  147.883931   
3       WeightedEnsemble_L2   0.738394     roc_auc      18.383380   69.473045   
4         LightGBMXT_BAG_L1   0.738130     roc_auc       1.733899    9.755707   
5           LightGBM_BAG_L2   0.738057     roc_auc     162.404857  118.008112   
6         LightGBMXT_BAG_L2   0.738045     roc_auc     162.536890  118.184644   
7           LightGBM_BAG_L1   0.737600     roc_auc       1.128252    8.740956   
8   RandomForestEntr_BAG_L1   0.728154     roc_auc      15.488222   45.790831   
9   RandomForestGini_BAG_L1   0.727166     roc_auc      15.327763   45.079168   
10    KNeighborsUnif_BAG_L1   0.651183     roc_auc      63.333550    0.409091   
11    KNeighborsDist_BAG_L1 

In [6]:
# predictor.feature_importance(train_data)

In [10]:
# 최적의 모델 가져오기
model_to_use = predictor.model_best

# 확률 예측
prob_predictions = predictor.predict_proba(test_data, model=model_to_use)

In [11]:
# 예측 결과를 test_data에 추가
test_data['probability'] = prob_predictions.iloc[:, 1]

# 최종 제출 파일 생성
submission = test_data[['ID', 'probability']]
submission = submission.sort_values(by='ID')

# 제출 파일 저장
submission.to_csv('../submission/code53_IVF_lgbm.csv', index=False, encoding='utf-8')

# 예측 결과 확인
print(submission.head())

           ID  probability
0  TEST_00000     0.001028
1  TEST_00001     0.000996
2  TEST_00002     0.144262
3  TEST_00003     0.083543
4  TEST_00004     0.451441


데이콘 PUBLIC xx

----

.